In [16]:
import tensorflow as tf
from tensorflow.keras import datasets, preprocessing, models, layers, optimizers, losses
import matplotlib.pyplot as plt
import numpy as np
import time

In [17]:
# Download Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# Read and decode text
text = open(path_to_file, 'rb').read().decode(encoding='utf8')
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [18]:
# Check unique characters
vocab = sorted(set(text))

# Create mapping for unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[:250])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0]


In [19]:
# Set maximum sequence length
seq_len = 100
examples_per_epoch = len(text)

# Create training examples and targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Convert sequence of characters to sequences of set seq_length
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
# For each sequence, duplicate and shift it to form the input and target text
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input example: {}".format(repr(''.join(idx2char[input_example]))))
    print("Target example: {}".format(repr(''.join(idx2char[target_example]))))

Input example: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target example: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [21]:
# Create training batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [22]:
# Build model
vocab_size = len(vocab)
embedding_dimension = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dimension, rnn_units, batch_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dimension, 
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, 
                            stateful=True,recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)])
    
    return model

model = build_model(vocab_size, embedding_dimension, rnn_units, BATCH_SIZE)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [23]:
import os
# Train model
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Configure checkpoints to ensure that checkpoints are saved during training
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 333s 2s/step - loss: 2.6814
Epoch 2/10
172/172 [==============================] - 311s 2s/step - loss: 1.9680
Epoch 3/10
172/172 [==============================] - 721s 4s/step - loss: 1.7028
Epoch 4/10
172/172 [==============================] - 317s 2s/step - loss: 1.5529
Epoch 5/10
172/172 [==============================] - 308s 2s/step - loss: 1.4629
Epoch 6/10
172/172 [==============================] - 1836s 11s/step - loss: 1.4018
Epoch 7/10
172/172 [==============================] - 306s 2s/step - loss: 1.3560
Epoch 8/10
172/172 [==============================] - 306s 2s/step - loss: 1.3176
Epoch 9/10
172/172 [==============================] - 307s 2s/step - loss: 1.2819
Epoch 10/10
172/172 [==============================] - 307s 2s/step - loss: 1.2499


In [27]:
# Save model
model.save("character-prediction.h5")

In [31]:
# Rebuild model with batch size = 1
import warnings
warnings.filterwarnings('ignore')

model = build_model(vocab_size, embedding_dimension, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_8 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_8 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
